# 04 - Evaluación y Análisis de Modelos

Este notebook profundiza en la evaluación de modelos con métricas avanzadas.

**Objetivos:**
- Métricas detalladas de clasificación
- Curvas ROC y Precision-Recall
- Matriz de confusión
- Feature importance
- Análisis de errores

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_curve, auc,
    precision_recall_curve, average_precision_score
)

sns.set_style('whitegrid')
print("✅ Librerías cargadas")

## 1. Carga de Modelo y Datos

In [ ]:
# Cargar mejor modelo
model = joblib.load('./models/best_model.joblib')
print(f"✅ Modelo cargado: {type(model).__name__}")

# Cargar datos de test
X_test = pd.read_parquet('./data/X_test.parquet')
y_test = pd.read_parquet('./data/y_test.parquet')['target']

print(f"Test set: {X_test.shape}")

## 2. Predicciones

In [ ]:
# Obtener predicciones
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print(f"Predicciones generadas: {len(y_pred)}")

## 3. Classification Report Detallado

💡 **Copilot tip:** Analiza precision y recall según tu caso de uso (ej: priorizar recall en detección de fraude).

In [ ]:
# Reporte completo
print("📊 Classification Report:")
print("=" * 60)
print(classification_report(y_test, y_pred, target_names=['Clase 0', 'Clase 1']))

## 4. Matriz de Confusión

In [ ]:
# Calcular matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Visualizar
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=['Clase 0', 'Clase 1'],
    yticklabels=['Clase 0', 'Clase 1'],
    cbar_kws={'label': 'Frecuencia'}
)
plt.title('Matriz de Confusión', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Valor Real', fontsize=12)
plt.xlabel('Valor Predicho', fontsize=12)
plt.tight_layout()
plt.show()

# Métricas derivadas
tn, fp, fn, tp = cm.ravel()
print(f"\n📈 Métricas de la Matriz de Confusión:")
print(f"   True Negatives (TN): {tn}")
print(f"   False Positives (FP): {fp}")
print(f"   False Negatives (FN): {fn}")
print(f"   True Positives (TP): {tp}")
print(f"   Specificity: {tn / (tn + fp):.4f}")
print(f"   Sensitivity (Recall): {tp / (tp + fn):.4f}")

## 5. Curvas ROC y Precision-Recall

In [ ]:
# Calcular curvas
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

precision, recall, _ = precision_recall_curve(y_test, y_proba)
avg_precision = average_precision_score(y_test, y_proba)

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ROC Curve
axes[0].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
axes[0].set_xlim([0.0, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].set_xlabel('False Positive Rate', fontsize=12)
axes[0].set_ylabel('True Positive Rate', fontsize=12)
axes[0].set_title('ROC Curve', fontsize=14, fontweight='bold')
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)

# Precision-Recall Curve
axes[1].plot(recall, precision, color='green', lw=2, label=f'PR curve (AP = {avg_precision:.3f})')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('Recall', fontsize=12)
axes[1].set_ylabel('Precision', fontsize=12)
axes[1].set_title('Precision-Recall Curve', fontsize=14, fontweight='bold')
axes[1].legend(loc='lower left')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Feature Importance

In [ ]:
# Obtener importancias
if hasattr(model, 'feature_importances_'):
    importances = pd.DataFrame({
        'feature': X_test.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Top 15 features
    top_features = importances.head(15)
    
    plt.figure(figsize=(10, 8))
    plt.barh(range(len(top_features)), top_features['importance'], color='steelblue')
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Importancia', fontsize=12)
    plt.title('Top 15 Features Más Importantes', fontsize=14, fontweight='bold', pad=20)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Top 10 Features:")
    for idx, row in importances.head(10).iterrows():
        print(f"   {row['feature']}: {row['importance']:.4f}")
else:
    print("⚠️  Modelo no tiene feature importances")

## 7. Análisis de Errores

In [ ]:
# Identificar errores
errors_df = X_test.copy()
errors_df['y_true'] = y_test.values
errors_df['y_pred'] = y_pred
errors_df['y_proba'] = y_proba
errors_df['error'] = errors_df['y_true'] != errors_df['y_pred']

# Estadísticas de errores
n_errors = errors_df['error'].sum()
error_rate = n_errors / len(errors_df)

print(f"\n❌ Análisis de Errores:")
print(f"   Total errores: {n_errors} / {len(errors_df)}")
print(f"   Error rate: {error_rate:.2%}")

# Casos con mayor incertidumbre (probabilidad cercana a 0.5)
uncertain = errors_df[errors_df['y_proba'].between(0.4, 0.6)].sort_values('y_proba')
print(f"\n🤔 Casos inciertos (prob entre 0.4-0.6): {len(uncertain)}")

if len(uncertain) > 0:
    print("\nPrimeros 5 casos más inciertos:")
    display(uncertain[['y_true', 'y_pred', 'y_proba']].head())

## 8. Distribución de Probabilidades Predichas

In [ ]:
plt.figure(figsize=(12, 6))

# Histograma por clase
plt.hist(y_proba[y_test == 0], bins=50, alpha=0.6, label='Clase 0 (Real)', color='blue')
plt.hist(y_proba[y_test == 1], bins=50, alpha=0.6, label='Clase 1 (Real)', color='red')
plt.axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='Threshold = 0.5')
plt.xlabel('Probabilidad Predicha', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.title('Distribución de Probabilidades Predichas por Clase Real', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Resumen Final

**Métricas clave:**
- ✅ AUC-ROC: Mide capacidad de discriminación
- ✅ Precision-Recall: Útil en datasets desbalanceados
- ✅ Feature Importance: Interpretabilidad del modelo
- ✅ Análisis de errores: Identificar patrones de fallo

**Próximo paso:** Despliegue y monitorización (notebook 05)